<a href="https://colab.research.google.com/github/AdityaRaj23/MeloGen/blob/main/MeloGen(updated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions:



1.   upload .krn file in your_music(some starting sounds)
2.   Run the Cells below
3.   **mel.mid** File will be created its the generated melody in midi format





In [2]:
!pip install music21

In [4]:
import os
import music21 as m21
import json
import tensorflow.keras as keras
import numpy as np

In [5]:
KERN_DATASET_PATH = "/content/your_music"
ACCEPTABLE_DURATIONS = [
    0.25,
    0.5,
    0.75,
    1.0,
    1.5,
    2,
    3,
    4

]
SAVE_DIR = "/content/dataset"
SINGLE_FILE_DATASET = "file_dataset"
SEQUENCE_LENGTH = 64
MAPPING_PATH = "mapping.json"

In [6]:
def load_songs_in_kern(dataset_path):
  #go through all the files in dataset and load them with music21
  songs = []
  for path, subdirs, files in os.walk(dataset_path):
    for file in files:
      if file[-3:] == "krn":
        song = m21.converter.parse(os.path.join(path,file))
        songs.append(song)
  return songs


In [7]:
def has_acceptable_duration(song,acceptable_durations):
  for note in song.flat.notesAndRests:
    if note.duration.quarterLength not in acceptable_durations:
      return False

  return True

In [8]:
def transpose(song):
  #get key from the song
  parts = song.getElementsByClass(m21.stream.Part)
  measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
  key = measures_part0[0][4]
  #estimate key using music21
  if not isinstance(key,m21.key.Key):
    key = song.analyze("key")
  #get interval for transposition E.g. Bmaj -> cmaj
  if key.mode == "major":
    interval = m21.interval.Interval(key.tonic , m21.pitch.Pitch("C"))
  elif key.mode == "minor":
    interval = m21.interval.Interval(key.tonic , m21.pitch.Pitch("A"))
  #transpose song by calculated interval
  transposed_song = song.transpose(interval)
  return transposed_song

In [9]:
def encode_song(song, time_step=0.25):
  """Converts a score into a time-series-like music representation. Each item in the encoded list represents 'min_duration'
  quarter lengths. The symbols used at each step are: integers for MIDI notes, 'r' for representing a rest, and '_'
  for representing notes/rests that are carried over into a new time step. Here's a sample encoding:

      ["r", "_", "60", "_", "_", "_", "72" "_"]

  :param song (m21 stream): Piece to encode
  :param time_step (float): Duration of each time step in quarter length
  :return:
  """

  encoded_song = []

  for event in song.flat.notesAndRests:

    # handle notes
    if isinstance(event, m21.note.Note):
        symbol = event.pitch.midi # 60
    # handle rests
    elif isinstance(event, m21.note.Rest):
        symbol = "r"

    # convert the note/rest into time series notation
    steps = int(event.duration.quarterLength / time_step)
    for step in range(steps):

      # if it's the first time we see a note/rest, let's encode it. Otherwise, it means we're carrying the same
      # symbol in a new time step
      if step == 0:
          encoded_song.append(symbol)
      else:
          encoded_song.append("_")

  # cast encoded song to str
  encoded_song = " ".join(map(str, encoded_song))

  return encoded_song

In [10]:
def preprocess(dataset_path):

  #load the folk song
  print("Loading Songs...")
  songs = load_songs_in_kern(dataset_path)
  print(f"loaded{len(songs)} songs.")

  for i, song in enumerate(songs):
    #filter out songs that have non-acceptable durations
    if not has_acceptable_duration(song,ACCEPTABLE_DURATIONS):
      continue
    #transpose songs to Cmaj/Amin
    song = transpose(song)
    #encode songs with music time series representation
    encoded_song = encode_song(song)
    # save songs to text file
    save_path = os.path.join(SAVE_DIR,str(i))
    with open(save_path,"w") as fp:
      fp.write(encoded_song)

In [11]:
def load(file_path):
  with open(file_path,"r") as fp:
    song = fp.read()

  return song

In [12]:
def create_single_file_dataset(dataset_path,file_dataset_path,sequence_length):
  new_song_delimiter = "/ " * sequence_length
  songs = ""
  #load encoded songs and add delimeters
  for path, _, files in os.walk(dataset_path):
    for file in files:
      file_path = os.path.join(path,file)
      song = load(file_path)
      songs = songs + song + " " + new_song_delimiter

  songs = songs[:-1]

  #save string that contains all the dataset

  with open(file_dataset_path ,"w") as fp:
    fp.write(songs)

  return songs


In [13]:
def convert_songs_to_int(songs):
  int_songs = []

  # load mappings
  with open(MAPPING_PATH,"r") as fp:
    mappings = json.load(fp)

  #cast songs string to a list
  songs = songs.split()

  #map songs to int
  for symbol in songs:
    int_songs.append(mappings[symbol])

  return int_songs


In [14]:
class MelodyGenerator:
    """A class that wraps the LSTM model and offers utilities to generate melodies."""

    def __init__(self, model_path="/content/melody.h5"):
        """Constructor that initialises TensorFlow model"""

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH


    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        """Generates a melody using the DL model and returns a midi file.

        :param seed (str): Melody seed with the notation used to encode the dataset
        :param num_steps (int): Number of steps to be generated
        :param max_sequence_len (int): Max number of steps in seed to be considered for generation
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.

        :return melody (list of str): List with symbols representing a melody
        """

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # (1, max_sequence_length, num of symbols in the vocabulary)
            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self._sample_with_temperature(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def _sample_with_temperature(self, probabilites, temperature):
        """Samples an index from a probability array reapplying softmax using temperature

        :param predictions (nd.array): Array containing probabilities for each of the possible outputs.
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.

        :return index (int): Selected output symbol
        """
        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index


    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):
        """Converts a melody into a MIDI file

        :param melody (list of str):
        :param min_duration (float): Duration of each time step in quarter length
        :param file_name (str): Name of midi file
        :return:
        """

        # create a music21 stream
        stream = m21.stream.Stream()

        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1

        # write the m21 stream to a midi file
        stream.write(format, file_name)



In [15]:
def main(length):
  preprocess(KERN_DATASET_PATH)
  songs = create_single_file_dataset(SAVE_DIR,SINGLE_FILE_DATASET,SEQUENCE_LENGTH)

  mg = MelodyGenerator()
  seed = songs[:15]
  melody = mg.generate_melody(seed, length, SEQUENCE_LENGTH, 0.3)
  mg.save_melody(melody)


In [17]:
#provide length as parameter to main
main(500)

Loading Songs...
loaded0 songs.
1/1 [==============================] - 0s 30ms/step


<ipython-input-14-dce89ea4b853>:76: RuntimeWarning: divide by zero encountered in log
  predictions = np.log(probabilites) / temperature


1/1 [==============================] - 0s 29ms/step
